## IMPORT LIBRARIES

In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# Visualization
import matplotlib.pyplot
import seaborn as sns
# Too see full dataframe...
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')



/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Libraries imported.


## WEBSCARP THE TABLE

In [17]:
path='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

r=requests.get(path)
soup=BeautifulSoup(r.text,'html.parser')


table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
 


# print(table_contents)
df=pd.DataFrame(table_contents)


df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head()






,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [18]:
df.rename(columns={'PostalCode':'Postalcode'},inplace=True)

In [19]:
df.shape

(103, 3)

In [5]:
!pip install geocoder

import geocoder # import geocoder


def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 10.8 MB/s eta 0:00:01


In [20]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_f3e17c7407554707bd99fe3eaf709699 = 'https://s3.eu.cloud-object-storage.appdomain.cloud'
else:
    endpoint_f3e17c7407554707bd99fe3eaf709699 = 'https://s3.private.eu.cloud-object-storage.appdomain.cloud'

client_f3e17c7407554707bd99fe3eaf709699 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='z3HykHm0EXDRI7Kjwj5o6x_ToVWAe6wY8RdoMK2NXx4q',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_f3e17c7407554707bd99fe3eaf709699)

body = client_f3e17c7407554707bd99fe3eaf709699.get_object(Bucket='torontoneighbourhood-donotdelete-pr-yoqnb07wi7en44',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
df_data_1.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(df_data_1, df, on='Postalcode')


In [22]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

In [23]:
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [24]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
40,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923


In [12]:
toronto_data=toronto_data.drop(['Borough'], axis = 1)

In [25]:
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
40,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923


In [26]:
# save the DataFrame as CSV file
toronto_data.to_csv("toronto_data.csv", index=False)

## Create a map of Torornto with Neighborhoods 

In [29]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


##  Create map of Toronto using latitude and longitude 

In [30]:

map_tor = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_tor)  
    
map_tor

In [31]:
# save the map as HTML file
map_tor.save('map_tor.html')

## Use the Foursquare API to explore the Neighborhoods

In [32]:
CLIENT_ID = 'MYZQXXWCNEX42ROETWXQ2O2B5QIZCT0U2LCVXDZCICNZ3NNJ' # your Foursquare ID
CLIENT_SECRET = 'ETTLK2N3GRJKYBJLGKO04TG4LOGJQAPYZ2DWK4WMGQBZC5US' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: MYZQXXWCNEX42ROETWXQ2O2B5QIZCT0U2LCVXDZCICNZ3NNJ
CLIENT_SECRET:ETTLK2N3GRJKYBJLGKO04TG4LOGJQAPYZ2DWK4WMGQBZC5US


## 100 venues that are within a radius of 2000 meters.

In [33]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [34]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3843, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
1,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
2,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
3,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
4,The Beaches,43.676357,-79.293031,Bagels On Fire,43.672864,-79.286784,Bagel Shop


## check how many venues were returned for each Neighorhood

In [35]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",100,100,100,100,100,100
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100
Davisville North,100,100,100,100,100,100



## Find out how many unique categories can be curated from all the venues.

In [36]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 223 uniques categories.


# Print out the list of categories.

In [37]:
venues_df['VenueCategory'].unique()[:50]

array(['Gastropub', 'Bakery', 'Indie Movie Theater',
       'Vegetarian / Vegan Restaurant', 'Bagel Shop', 'Coffee Shop',
       'Ice Cream Shop', 'French Restaurant', 'Park',
       'Japanese Restaurant', 'Beach', 'Cupcake Shop', 'Tea Room',
       'Mexican Restaurant', 'Pub', 'Middle Eastern Restaurant',
       'Breakfast Spot', 'Thai Restaurant', 'Hungarian Restaurant',
       'Burger Joint', 'Sandwich Place', 'Diner', 'BBQ Joint',
       'Skating Rink', 'Café', 'Chocolate Shop', 'Bar', 'Beer Store',
       'Playground', 'Caribbean Restaurant', 'Pharmacy',
       'Health Food Store', 'Pizza Place', 'Gas Station', 'Bank',
       'Butcher', 'Fish & Chips Shop', 'Sushi Restaurant',
       'Asian Restaurant', 'Ramen Restaurant', 'Gym', 'Grocery Store',
       'Hardware Store', 'Food & Drink Shop',
       'Paper / Office Supplies Store', 'Train Station', 'Restaurant',
       'American Restaurant', 'Ethiopian Restaurant',
       'Turkish Restaurant'], dtype=object)

## Analyze Each Neighborhood.

In [38]:
# one hot encoding
tor_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

print(tor_onehot.shape)
tor_onehot.head()

(3843, 224)


,Neighborhoods,Accessories Store,Airport,American Restaurant,Amphitheater,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Beach,Beer Bar,Beer Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Nudist Beach,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Zoo
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [39]:
tor_grouped = tor_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(tor_grouped.shape)
tor_grouped

(39, 224)


,Neighborhoods,Accessories Store,Airport,American Restaurant,Amphitheater,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Beach,Beer Bar,Beer Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Nudist Beach,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Zoo
0,Berczy Park,0.00,0.00,0.010000,0.00,0.00,0.02,0.00,0.000000,0.010000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.00,0.00,0.030000,0.000000,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.050000,0.00,0.00,0.00,0.00,0.010000,0.01,0.01,0.00,0.010000,0.010000,0.100000,0.00,0.00,0.000000,0.00,0.000000,0.020000,0.01,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.02,0.020000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.030000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.01,0.01,0.00,0.01,0.00,0.010000,0.00,0.00,0.010000,0.00,0.01,0.00,0.00,0.00,0.00,0.010000,0.020000,0.010000,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.03,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.020000,0.030000,0.000000,0.000000,0.00,0.01,0.00,0.01,0.00,0.00,0.02,0.0

In [43]:
len(tor_grouped[tor_grouped["Ice Cream Shop"] > 0])

27

## Create a new DataFrame for Ice cream data only.

In [40]:
tor_Ice  = tor_grouped[["Neighborhoods","Ice Cream Shop"]]

In [41]:
tor_Ice.head()

,Neighborhoods,Ice Cream Shop
0,Berczy Park,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.01
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.00
3,Central Bay Street,0.01
4,Christie,0.03


 ## Cluster Neighborhoods -Run k-means to cluster the neighborhoods in torornto into 3 clusters.

In [42]:
# set number of clusters
kclusters = 3

tor_clustering = tor_Ice.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 2, 1, 0, 0, 2, 1, 1, 1], dtype=int32)

In [43]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
tor_merged = tor_Ice.copy()

# add clustering labels
tor_merged["Cluster Labels"] = kmeans.labels_

In [44]:
tor_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
tor_merged.head()

,Neighborhood,Ice Cream Shop,Cluster Labels
0,Berczy Park,0.00,2
1,"Brockton, Parkdale Village, Exhibition Place",0.01,1
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,2
3,Central Bay Street,0.01,1
4,Christie,0.03,0


In [45]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(toronto_data.set_index("Neighborhood"), on="Neighborhood")

print(tor_merged.shape)
tor_merged.head() # check the last columns!

(39, 7)


,Neighborhood,Ice Cream Shop,Cluster Labels,Postalcode,Borough,Latitude,Longitude
0,Berczy Park,0.00,2,M5E,Downtown Toronto,43.644771,-79.373306
1,"Brockton, Parkdale Village, Exhibition Place",0.01,1,M6K,West Toronto,43.636847,-79.428191
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,2,M5V,Downtown Toronto,43.628947,-79.394420
3,Central Bay Street,0.01,1,M5G,Downtown Toronto,43.657952,-79.387383
4,Christie,0.03,0,M6G,Downtown Toronto,43.669542,-79.422564


In [51]:
# sort the results by Cluster Labels
print(tor_merged.shape)
tor_merged.sort_values(["Cluster Labels"], inplace=True)
tor_merged

(39, 5)


,Neighborhood,Ice Cream Shop,Cluster Labels,Latitude,Longitude
38,"University of Toronto, Harbord",0.020000,0,43.662696,-79.400049
36,"The Danforth West, Riverdale",0.020000,0,43.679557,-79.352188
35,The Danforth East,0.030000,0,43.685347,-79.338106
4,Christie,0.030000,0,43.669542,-79.422564
5,Church and Wellesley,0.020000,0,43.665860,-79.383160
33,"The Annex, North Midtown, Yorkville",0.030000,0,43.672710,-79.405678
32,"Summerhill West, Rathnelly, South Hill, Forest...",0.030000,0,43.686412,-79.400049
16,"High Park, The Junction South",0.030000,0,43.661608,-79.464763
23,"Parkdale, Roncesvalles",0.020000,0,43.648960,-79.456325
13,Forest Hill North & West,0.020000,0,43.696948,-79.411307


Finally, let's visualize the resulting clusters

# create map

In [46]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [47]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Examine Clusters

In [49]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0]

,Neighborhood,Ice Cream Shop,Cluster Labels,Postalcode,Borough,Latitude,Longitude
4,Christie,0.03,0,M6G,Downtown Toronto,43.669542,-79.422564
5,Church and Wellesley,0.02,0,M4Y,Downtown Toronto,43.665860,-79.383160
13,Forest Hill North & West,0.02,0,M5P,Central Toronto,43.696948,-79.411307
16,"High Park, The Junction South",0.03,0,M6P,West Toronto,43.661608,-79.464763
23,"Parkdale, Roncesvalles",0.02,0,M6R,West Toronto,43.648960,-79.456325
26,Rosedale,0.02,0,M4W,Downtown Toronto,43.679563,-79.377529
32,"Summerhill West, Rathnelly, South Hill, Forest...",0.03,0,M4V,Central Toronto,43.686412,-79.400049
33,"The Annex, North Midtown, Yorkville",0.03,0,M5R,Central Toronto,43.672710,-79.405678
35,The Danforth East,0.03,0,M4J,East York/East Toronto,43.685347,-79.338106
36,"The Danforth West, Riverdale",0.02,0,M4K,East Toronto,43.679557,-79.352188


In [50]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1]

,Neighborhood,Ice Cream Shop,Cluster Labels,Postalcode,Borough,Latitude,Longitude
1,"Brockton, Parkdale Village, Exhibition Place",0.010000,1,M6K,West Toronto,43.636847,-79.428191
3,Central Bay Street,0.010000,1,M5G,Downtown Toronto,43.657952,-79.387383
7,Davisville,0.010000,1,M4S,Central Toronto,43.704324,-79.388790
8,Davisville North,0.010000,1,M4P,Central Toronto,43.712751,-79.390197
9,"Dufferin, Dovercourt Village",0.014286,1,M6H,West Toronto,43.669005,-79.442259
10,Enclave of M4L,0.010000,1,M7Y,East Toronto Business,43.662744,-79.321558
14,"Garden District, Ryerson",0.010000,1,M5B,Downtown Toronto,43.657162,-79.378937
19,Lawrence Park,0.013699,1,M4N,Central Toronto,43.728020,-79.388790
20,"Little Portugal, Trinity",0.010000,1,M6J,West Toronto,43.647927,-79.419750
22,North Toronto West,0.010000,1,M4R,Central Toronto,43.715383,-79.405678


In [51]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2]

,Neighborhood,Ice Cream Shop,Cluster Labels,Postalcode,Borough,Latitude,Longitude
0,Berczy Park,0.0,2,M5E,Downtown Toronto,43.644771,-79.373306
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,2,M5V,Downtown Toronto,43.628947,-79.394420
6,"Commerce Court, Victoria Hotel",0.0,2,M5L,Downtown Toronto,43.648198,-79.379817
11,Enclave of M5E,0.0,2,M5W,Downtown Toronto Stn A,43.646435,-79.374846
12,"First Canadian Place, Underground city",0.0,2,M5X,Downtown Toronto,43.648429,-79.382280
15,"Harbourfront East, Union Station, Toronto Islands",0.0,2,M5J,Downtown Toronto,43.640816,-79.381752
17,"India Bazaar, The Beaches West",0.0,2,M4L,East Toronto,43.668999,-79.315572
18,"Kensington Market, Chinatown, Grange Park",0.0,2,M5T,Downtown Toronto,43.653206,-79.400049
21,"Moore Park, Summerhill East",0.0,2,M4T,Central Toronto,43.689574,-79.383160
25,"Richmond, Adelaide, King",0.0,2,M5H,Downtown Toronto,43.650571,-79.384568



Observations:
Most of the icecream shops are concentrated in the central area of Toronto, with the highest number in cluster 0 and moderate number in cluster 1. On the other hand, cluster 2 has very low number of ice cream shops in the neighborhoods. This represents a great opportunity and high potential areas to open new ice cream shops as there is very little to no competition from existing.  Therefore, this study recommends the ice crem brand to capitalize on these findings to open their shops in neighborhoods in cluster 2 with little to no competition.